## 1. Data Preprocessing

### （1）图片加载、转化及存储

### 类型一
* 数据的组织结构是以若干个文件夹进行的， 每类图片所存储的文件夹的名字使用字符命名。 比如，文件夹sub_dir1是一个类型，里面存储的所有图片均是一类；文件夹sub_dir2是一个类型，里面存储的所有图片均是一类；依次类推。
* 读取图片和标签时事先以字典的方式指定所有图片所属的类：key对应于标签类别，vallue对应于每类图片所在的子目录
* 标签处理成int型是为了后续的one-hot编码之用

把若干个文件夹下的图片所在的路径存入列表：

 ROOT_DIR
    |---sub_dir0
        |---image0.jpg
        |---image1.jpg
        | .
    |---sub_dir1
    |---sub_dir2
    | .

In [ ]:
# 类型一的图片处理方法

# 把若干个文件夹下的图片所在的路径存入列表：
import glob

ROOT_DIR = '...../'
sub_dir_dic = {0: 'sub_dir0', 1: 'sub_dir1', 2: 'sub_dir2'}
for idx, sub_dir in sub_dir_dic.items():
    img_paths = [k for k in glob.glob(ROOT_DIR + '/' + sub_dir + '/*')]

In [ ]:
# 根据所有图片的路径制作数据和标签
import cv2 
import numpy as np

def imgs_and_labels():
    imgs = []
    labels = []
    
    img_height = 40
    img_width = 30
    
    
    for idx, sub_dir in sub_dir_dic.items():
        label = idx
         
        img_paths = [k for k in glob.glob(ROOT_DIR + '/' + sub_dir + '/*')]
        
        for i, path in enumerate(img_paths):
            img = cv2.imread(path) # BGR
            
            # BGR->RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Resize
            img = cv2.resize(img, (img_height, img_width))
            
            imgs.append(img)
            labels.append(label)
            
            # convert to numpy array
            imgs = np.array(imgs)
            labels = np.array(labels)
            
            return imgs, labels

### 类型二
* 数据的组织结构是以若干个文件夹进行的， 每类图片所存储的文件夹的名字使用数字命名。比如，文件夹0000是一个类型，里面存储的所有图片均是一类；文件夹0002是一个类型，里面存储的所有图片均是一类；依次类推。
* 读取图片的方式是以一次性读取所有类别的图片，标签是根据图片的路径倒数第二项值确定的。比如，int(PurePath(img_path).parts[-2])命令可以把某类文件夹的名称转化为整数并用作后续的one-hot编码

 ROOT_DIR
    |---0000
        |---image0.jpg
        |---image1.jpg
        | .
    |---0001
    |---0002
    | .

In [ ]:
# 类型二的图片处理方法
import os
from pathlib import PurePath
from skimage import io

try:
    # 从已有的h5py文件中加载数据
    with h5py.File('dataset.h5') as h5f:
        X, y = h5f['imgs'][:], h5f['labels'][:]
        
except(IOError, OSError, KeyError):
    # 从原始图片中读取并转换数据
    ROOT_DIR = '......'
    imgs = []
    labels = []
    
    # 获取ROOT_DIR下的所有子目录下的图片的路径, 第一个*号代表所有子目录
    # 第二个*代表某个子目录下的所有图片的名称
    all_img_paths = glob.glob(os.path.join(ROOT_DIR, '*/*.ppm'))
    
    # 把所有图片的路径打乱
    np.random.shuffle(all_img_paths)
    
    for img_path in all_img_paths:
        try:
            img = io.imread(img_path)
            label = int(PurePath(img_path).parts[-2])
            
            imgs.append(img)
            labels.append(label)
        except(IOError):
            pass
    
    X = np.array(imgs, dtype='float32')
    y = np.eye(NUM_CLASSES, dtype='uint8')[labels]
    
    # 把转换后的图片数据和标签存储为h5py文件
    with h5py.File('dataset.h5', 'w') as h5f:
        h5f.create_dataset('imgs', data=X)
        h5f.create_dataset('labels', data=y)

In [ ]:
# 把数据和标签存储为h5py文件或从h5py文件中加载数据和标签
import h5py
import keras
from sklearn.model_selection import train_test_split

NUM_CLASSES = 43

def get_dataset(save=False, load=False):
    if load: # 如果已有数据和标签的h5py文件,则直接从h5py文件加载
        # 数据集
        h5f = h5py.File('dataset.h5', 'r')
        X_train = h5f['X_train'][:] # 训练集
        X_test = h5f['X_test'][:]   # 验证集
        h5f.close()
        # 以上方法也可以使用如下方法实现
        #with h5py.File('dataset.h5', 'r') as h5f:
        #    X_train, X_test = h5f['X_train'][:], h5f['X_test'][:]
        
        # 标签
        h5f = h5py.File('labels.h5', 'r')
        y_train = h5f['y_train'][:] # 训练集的标签
        y_test = h5f['y_test'][:]   # 验证集的标签
        h5f.close()       
        # 以上方法也可以使用如下方法实现
        #with h5py.File('labels', 'r') as h5f:
        #    y_train, y_test = h5f['y_train'][:], h5f['y_test'][:]
    else:
        # 从原始图片数据处理
        X, y = imgs_and_labels() # X和y已经是numpy形式的数据和标签
        
        # 对标签进行 one-hot 编码
        y = keras.utils.to_categorical(y, num_classes=NUM_CLASSES)
        
        # 把数据分为训练数据和验证数据(70% vs 30%)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        
        #保存为h5py文件
        if save:
            # data
            h5f = h5py.File('dataset.h5', 'w')
            h5f.create_dataset('X_train', data=X_train)
            h5f.create_dataset('X_test', data=X_test)
            h5f.close()         
            # 以上方法也可以使用如下方法实现
            # with h5py.File('dataset.h5', 'w') as h5f:
            #     h5f.create_dataset('X_train', data=X_train)
            #     h5f.create_dataset('X_test', data=X_test)
            
            # labels
            h5f = h5py.File('labels.h5', 'w')
            h5f.create_dataset('y_train', data=y_train)
            h5f.create_dataset('y_test', data=y_test)
            h5f.close()
            # 以上方法也可以使用如下方法实现
            # with h5py.File('labels.h5', 'w') as h5f:
            #     h5f.create_dataset('y_train', data=y_train)
            #     h5f.create_dataset('y_test', data=y_test)
                
            
    # 对图片像素进行归一化处理
    X_train = X_train.astype('float32') / 255.
    X_test = X_test.astype('float32') / 255.
    print('Train data shape: ', X_train.shape)
    print('Train labels shape: ', y_train.shape)
    print('Test data shape: ', X_test.shape)
    print('Test labels shape:, ', y_test.shape)
    
    return X_train, X_test, y_train, y_test

### 类型三
* 图片数据是以train和test两个子目录组成的，类别标签是以.csv格式存储的。

 ROOT_DIR
    |---train
    |   |---0000.jpg
    |   |---0001.jpg
    |   | .
    |
    |---test
    |   |---2333.jpg
    |   |---2334.jpg
    |   |.
    | 
    |---labels.csv

In [ ]:
# 图片数据处理

from natsort import natsorted
import cv2

# 数据所在基础目录
ROOT_DIR = '..../data'

# 把图像转换成的目标大小
img_height, img_width = 32, 32

# 转换后的图片存储的目录

train_data_dir = ROOT_DIR + '/train_processed' 
test_data_dir = ROOT_DIR + '/test_precessed'

# 创建目录
if not os.path.exists(train_data_dir):
    os.makedirs(train_data_dir)
    
if not os.path.exists(test_data_dir):
    os.makedirs(test_data_dir)
    
# 处理图片大小与色彩
for datatype in ['train', 'test']:
    # 获取train(test)目录下的所有图片的路径
    img_paths = natsorted(glob.glob(ROOT_DIR + '/' + datatype + '/*'))
    
    # 以numpy数据存储train(test)目录下的所有图片
    imgs_data = np.zeros((len(img_paths), img_height, img_width))
    
    for i, img_path in enumerate(img_paths):
        img = cv2.imread(img_path, 0) # 以灰度图像读取
        img = cv2.resize(img, (img_height, img_width))
        
        imgs_data[i] = img
        
        # 同时把处理后的图片存储到文件夹内
        img_name = os.path.basename(img_path)
        img_name_dot_split = img_name.split('.')
        new_img_name = str(int(img_name_dot_split[0])).zfill(5) + '.'\
                        + img_name_dot_split[-1].lower()
        new_img_path = ROOT_DIR + '/' + datatype + 'preprocessed' + '/' + new_img_name
        cv2.imwrite(new_img_path, img)
     
    # 扩充维度
    imgs_data = imgs_data[:, :, :, np.newaxis]
    
    # 归一化
    imgs_data = imgs_data.astype('floate32') / 255.
    
    # 存储为numpy格式
    np.save(ROOT_DIR + '/' + datatype + 'processed' + '.npy', imgs_data)


In [ ]:
# 标签处理

import pandas as pd

def label2int(label):
    
    
    # 把某个标签转换成唯一的整数
    #int_lable = f(label)
    
    
    return int_label

y_train = pd.read_csv(ROOT_DIR, 'labels.csv').values[:, 1] # 标签所在的列

# 假设标签类别为62
Y_train = np.zeros((y_train.shape[0], 62)) # 标签one-hot编码矩阵：行数为样本数，列数为类别数

for i in range(y_train.shape[0]):
    Y_train[i][label2int(y_train[i])] = 1 # one-hot 编码